In [135]:
import pandas as pd

df = pd.read_csv("datasets/Train.csv")

In [136]:
df.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [137]:
# df['label'] = df['label'].map({1: 'positive', 0: 'negative'}).astype('object')

In [138]:
df

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
...,...,...
39995,"""Western Union"" is something of a forgotten cl...",1
39996,This movie is an incredible piece of work. It ...,1
39997,My wife and I watched this movie because we pl...,0
39998,"When I first watched Flatliners, I was amazed....",1


In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    40000 non-null  object
 1   label   40000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 625.1+ KB


In [140]:
df.describe()

,label
count,40000.000000
mean,0.499525
std,0.500006
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [141]:
df.duplicated().sum()

np.int64(277)

In [142]:
df.drop_duplicates()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
...,...,...
39995,"""Western Union"" is something of a forgotten cl...",1
39996,This movie is an incredible piece of work. It ...,1
39997,My wife and I watched this movie because we pl...,0
39998,"When I first watched Flatliners, I was amazed....",1


In [143]:
df.isnull().sum()

text     0
label    0
dtype: int64

In [144]:
df[df['label'] == 'positive'].shape[0], df[df['label'] == 'negative'].shape[0]

(0, 0)

In [ ]:
import re, emoji, html
from bs4 import BeautifulSoup

def clean(text):
    text = BeautifulSoup(text, "html.parser").get_text(" ")  # strip HTML tags
    text = html.unescape(text)                               # e.g. &amp; → &
    text = emoji.demojize(text, delimiters=(" ", " "))       # 😀 →  smile_face 
    text = re.sub(r"\s+", " ", text).strip()                 # collapse spaces
    return text

df["clean_text"] = df["text"].astype(str).apply(clean)
df = df.drop_duplicates(subset="clean_text")
df = df[~df["clean_text"].str.len().lt(5)]                 # drop ultra-short

In [146]:
df.describe()

,label
count,39722.000000
mean,0.501183
std,0.500005
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [147]:
df_test = pd.read_csv("datasets/Test.csv")

In [148]:
# df_test['label'] = df_test['label'].map({1: 'positive', 0: 'negative'}).astype('object')

In [149]:
df_test

,text,label
0,I always wrote this series off as being a comp...,0
1,1st watched 12/7/2002 - 3 out of 10(Dir-Steve ...,0
2,This movie was so poorly written and directed ...,0
3,The most interesting thing about Miryang (Secr...,1
4,"when i first read about ""berlin am meer"" i did...",0
...,...,...
4995,This is the kind of picture John Lassiter woul...,1
4996,A MUST SEE! I saw WHIPPED at a press screening...,1
4997,NBC should be ashamed. I wouldn't allow my chi...,0
4998,This movie is a clumsy mishmash of various gho...,0


In [150]:
df_test["clean_text"] = df_test["text"].astype(str).apply(clean)
df_test = df_test.drop_duplicates(subset="clean_text")
df_test = df_test[~df_test["clean_text"].str.len().lt(5)]  

In [151]:
df_valid = pd.read_csv("datasets/Valid.csv")

In [152]:
# df_valid['label'] = df_valid['label'].map({1: 'positive', 0: 'negative'}).astype('object')

In [153]:
df_valid["clean_text"] = df_valid["text"].astype(str).apply(clean)
df_valid = df_valid.drop_duplicates(subset="clean_text")
df_valid = df_valid[~df_valid["clean_text"].str.len().lt(5)]  

In [154]:
df_test.describe()

,label
count,4995.000000
mean,0.500901
std,0.500049
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [155]:
df_valid.describe()

,label
count,4998.000000
mean,0.503001
std,0.500041
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [156]:
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained("bert-base-cased")


In [157]:
# Check text lengths after tokenization
sample_texts = df["clean_text"].head(100).tolist()
token_lengths = [len(tok.encode(text)) for text in sample_texts]
print(f"Max tokens: {max(token_lengths)}")
print(f"Texts truncated: {sum(1 for length in token_lengths if length > 512)}")

Token indices sequence length is longer than the specified maximum sequence length for this model (1334 > 512). Running this sequence through the model will result in indexing errors


Max tokens: 1334
Texts truncated: 12


In [ ]:
import torch

def tokenize(batch):
    encodings = tok(batch["clean_text"],
        padding="max_length",
        truncation=True,
        max_length=512)
    encodings['labels'] = batch['label']
    return encodings

ds = DatasetDict({
    "train": Dataset.from_pandas(df[["clean_text", "label"]]),
    "validation": Dataset.from_pandas(df_valid[["clean_text", "label"]]),
    "test": Dataset.from_pandas(df_test[["clean_text", "label"]]),
}).map(tokenize, batched=True).remove_columns(["clean_text", "label", "__index_level_0__"])

Map: 100%|██████████| 4995/4995 [00:01<00:00, 3070.36 examples/s]


TypeError: 'torch.dtype' object is not callable

In [159]:
print(ds["train"].features)

{'input_ids': List(Value('int32')), 'token_type_ids': List(Value('int8')), 'attention_mask': List(Value('int8')), 'labels': Value('int64')}


In [160]:
from transformers import (AutoModelForSequenceClassification,
                          TrainingArguments, Trainer)
# from evaluate import evaluate
import numpy as np
from sklearn.metrics import (accuracy_score, precision_recall_fscore_support)

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-cased",
    num_labels=2
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc  = accuracy_score(labels, preds)
    p, r, f, _ = precision_recall_fscore_support(labels, preds, average="macro")
    return {"accuracy": acc, "precision": p, "recall": r, "f1": f}

args = TrainingArguments(
    output_dir="models/bert-imdb-sentiment",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=15,
    warmup_steps=200,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    push_to_hub=False,
    logging_steps=100
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    compute_metrics=compute_metrics
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [161]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.233000,0.246592,0.930172,0.930186,0.930197,0.930172


KeyboardInterrupt: 